In [1]:
#nltk.download('stopwords')
#nltk.download('punkt')
#!python -m spacy download es_core_news_md
#!python -m spacy validate

import warnings
import pandas as pd
warnings.filterwarnings("ignore", category=DeprecationWarning)
pd.options.mode.chained_assignment = None  # default='warn'

from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
tqdm.pandas()
import numpy as np

from utils import clean_dataset_basedOn_media
from utils import cluster_by_month

In [2]:
df = pd.read_csv("../data/loslagos-comunas.csv")
df = cluster_by_month(clean_dataset_basedOn_media(df))
df.isna().any()

date               False
media_outlet       False
url                False
title              False
text               False
content            False
comuna              True
date_clustering    False
dtype: bool

In [3]:
df.date_clustering.value_counts()

2022-03    4617
2021-12    4492
2021-10    4370
2021-11    4326
2022-01    4182
2022-02    3950
2022-04    3384
Name: date_clustering, dtype: int64

In [4]:
# Obtenemos las etiquetas del value_counts 
months = df.date_clustering.value_counts().index.tolist()

# se hará un análisis del primer mes
import datetime
dates = [datetime.datetime.strptime(ts, "%Y-%m") for ts in months]
dates.sort()
sorteddates = [datetime.datetime.strftime(ts, "%Y-%m") for ts in dates]

In [5]:
##### Queremos seleccionar por comuna?
#selection = ['puerto montt']
#mask = df.comuna.apply(lambda x: any(item for item in selection if item in x))
#df1 = df[mask]

# [BERTopic](https://maartengr.github.io/BERTopic/index.html)
BERTopic is a topic modeling technique that leverages transformers and c-TF-IDF to create dense clusters allowing for easily interpretable topics whilst keeping important words in the topic descriptions.

In [6]:
#!pip install bertopic
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
selected = df[df.date_clustering == sorteddates[0]]
docs = selected.content.tolist()
topic_model = BERTopic(language="spanish",
                       calculate_probabilities=True,
                       verbose=True,
                       nr_topics=15)

topics, probs = topic_model.fit_transform(docs)

Batches:   0%|          | 0/137 [00:00<?, ?it/s]

2022-09-29 17:00:25,976 - BERTopic - Transformed documents to Embeddings
2022-09-29 17:01:05,720 - BERTopic - Reduced dimensionality
2022-09-29 17:01:08,469 - BERTopic - Clustered reduced embeddings
2022-09-29 17:01:16,054 - BERTopic - Reduced number of topics from 105 to 16


In [8]:
freq = topic_model.get_topic_info()
print(freq)

    Topic  Count                                            Name
0      -1   2825                    -1_años_personas_chile_parte
1       0    166                  0_festival_teatro_música_artes
2       1    155               1_torneo_deportes_provincial_colo
3       2    140                    2_colo_partido_puntos_equipo
4       3    111                 3_casos_nuevos_contagios_región
5       4    104                4_dosis_salud_vacunación_vacunas
6       5    103        5_educación_estudiantes_admisión_proceso
7       6     96                6_hospital_salud_mayores_adultos
8       7     95             7_bomberos_accidente_incendio_lugar
9       8     94                   8_sichel_kast_candidato_boric
10      9     92              9_roja_paraguay_selección_brereton
11     10     90                 10_tribunal_años_juicio_víctima
12     11     88          11_piñera_presidente_acusación_dominga
13     12     72  12_regional_recursos_desarrollo_organizaciones
14     13     70         

In [10]:
fig = topic_model.visualize_topics()
fig.write_image("../img/fig0411.png")

<img src="https://github.com/rickiwasho/proyecto-titulo/blob/main/img/fig0411.png?raw=true" />

In [11]:
labels = topic_model.generate_topic_labels()
print(labels)

['-1_años_personas_chile', '0_festival_teatro_música', '1_torneo_deportes_provincial', '2_colo_partido_puntos', '3_casos_nuevos_contagios', '4_dosis_salud_vacunación', '5_educación_estudiantes_admisión', '6_hospital_salud_mayores', '7_bomberos_accidente_incendio', '8_sichel_kast_candidato', '9_roja_paraguay_selección', '10_tribunal_años_juicio', '11_piñera_presidente_acusación', '12_regional_recursos_desarrollo', '13_joven_muerte_cuerpo', '14_mujer_carabineros_víctima']


In [12]:
topic_model.get_topic(0)  # Select the most frequent topic

[('festival', 0.03310644806723276),
 ('teatro', 0.02553253049535346),
 ('música', 0.024818606307270216),
 ('artes', 0.02086583709959196),
 ('cultural', 0.02011821746626615),
 ('arte', 0.019126513483863794),
 ('octubre', 0.016901393888092534),
 ('cultura', 0.01684748237058268),
 ('actividades', 0.015020860466265898),
 ('través', 0.014343368208111558)]

In [13]:
fig = topic_model.visualize_barchart()
fig.write_image("../img/fig0412.png")

<img src="https://github.com/rickiwasho/proyecto-titulo/blob/main/img/fig0412.png?raw=true" />

In [14]:
fig = topic_model.visualize_heatmap()
fig.write_image("../img/fig0413.png")

<img src="https://github.com/rickiwasho/proyecto-titulo/blob/main/img/fig0413.png?raw=true" />

In [15]:
fig = topic_model.visualize_hierarchy()
fig.write_image("../img/fig0414.png")

<img src="https://github.com/rickiwasho/proyecto-titulo/blob/main/img/fig0414.png?raw=true" />

In [16]:
fig = topic_model.visualize_documents(docs)
fig.write_image("../img/fig0415.png")

<img src="../img/fig0415.png"/>

In [17]:
labels=topic_model.generate_topic_labels()
labels

['-1_años_personas_chile',
 '0_festival_teatro_música',
 '1_torneo_deportes_provincial',
 '2_colo_partido_puntos',
 '3_casos_nuevos_contagios',
 '4_dosis_salud_vacunación',
 '5_educación_estudiantes_admisión',
 '6_hospital_salud_mayores',
 '7_bomberos_accidente_incendio',
 '8_sichel_kast_candidato',
 '9_roja_paraguay_selección',
 '10_tribunal_años_juicio',
 '11_piñera_presidente_acusación',
 '12_regional_recursos_desarrollo',
 '13_joven_muerte_cuerpo',
 '14_mujer_carabineros_víctima']

In [18]:
df['topic'] = ""
count = 0
for doc in docs:
    
    df.at[df.index[df['content'] == doc], 'topic'] = labels[topics[count]+1]
    count+=1

In [25]:
dfinv = df.loc[df.topic!="-1_años_personas_chile"]
dfinv = dfinv.loc[df.date_clustering=="2021-10"]
dfinv

,date,media_outlet,url,title,text,content,comuna,date_clustering,topic
4,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/galeria-de-ar...,Galería de Arte Machacoya realizará remate de ...,"Hoy viernes a las 18:30 horas, en Machacoya At...",galería arte machacoya realizará remate obras ...,NaN,2021-10,0_festival_teatro_música
7,2021-10-01,seminariolocal,https://www.semanariolocal.cl/archivos/2207,Gremio médico rechaza cierre de camas críticas...,El Capítulo Médico Ancud y el Consejo Regional...,gremio médico rechaza cierre camas críticas im...,"['ancud', 'puerto montt']",2021-10,6_hospital_salud_mayores
19,2021-10-01,elquellonino,https://elquellonino.cl/29951/exitoso-operativ...,Exitoso operativo de fisiatría se realizó en H...,Usuarios destacaron atenciones de especialista...,exitoso operativo fisiatría realizó hospital c...,['queilen'],2021-10,6_hospital_salud_mayores
20,2021-10-01,elquellonino,https://elquellonino.cl/29949/festival-cultura...,Festival “Cultura por Fibra” y II Encuentro Pr...,El Centro Cultural Municipal de Castro también...,festival “ cultura fibra ” ii encuentro provin...,['castro'],2021-10,0_festival_teatro_música
21,2021-10-01,elquellonino,https://elquellonino.cl/29946/comunidad-valora...,Comunidad valora compromiso y acciones impleme...,Director del servicio de salud en el encuentro...,comunidad valora compromiso acciones implement...,['queilen'],2021-10,6_hospital_salud_mayores
...,...,...,...,...,...,...,...,...,...
4372,2021-10-31,elvacanudo,https://www.elvacanudo.cl/noticia/sociedad/adu...,Adulto mayor queda en riesgo vital tras ser at...,"Durante la jornada de este sábado, un hombre d...",adulto mayor queda riesgo vital tras ser atrop...,NaN,2021-10,7_bomberos_accidente_incendio
4378,2021-10-31,radiosago,https://www.radiosago.cl/investigan-homicidio-...,Investigan homicidio de hombre en Panguipulli:...,La Policía de Investigaciones investiga un hom...,investigan homicidio hombre panguipulli autor ...,NaN,2021-10,13_joven_muerte_cuerpo
4381,2021-10-31,radiosago,https://www.radiosago.cl/cristian-romero-asume...,Cristián Romero asume como interino en la “U” ...,Universidad de Chile confirmó en forma oficial...,cristián romero asume interino “ u ” cinco nom...,NaN,2021-10,2_colo_partido_puntos
4383,2021-10-31,radiosago,https://www.radiosago.cl/encuesta-pulso-ciudad...,Encuesta Pulso Ciudadano: Kast encabeza las pr...,"Este domingo, durante el programa Mesa Central...",encuesta pulso ciudadano kast encabeza prefere...,NaN,2021-10,8_sichel_kast_candidato


In [26]:
dfinv.to_csv('../2021-10-bertopic.csv', index=False)

In [ ]:
#topic_model.reduce_topics(docs, topics, nr_topics=7)

In [ ]:
#topic_model.get_representative_docs() # entrega el doc más representativo para los 7 topicos